This code is for simulate unified random sampler distribution

In [1]:
# Python reference for a Unified Random Sampler (Uniform, Ternary, Discrete Gaussian via CDT)
# Author: Muhammad Ogin Hasanuddin
# Requirements: numpy

import math
import numpy as np
from dataclasses import dataclass

In [22]:
# ------------------------------
# Helpers (Barrett reduce, etc.)
# ------------------------------
def barrett_reduce(x: np.ndarray, q: int) -> np.ndarray:
    """
    Reference Barrett reduction (64-bit) for demonstration.
    In hardware you'd choose k so that 2^k > q^2. Here we use k=64.
    """
    k = 64
    mu = (1 << k) // q
    # Ensure x is unsigned 128-ish via Python big ints, apply vectorized formula
    t = ((x.astype(object) * mu) >> k).astype(object)
    r = (x.astype(object) - t * q).astype(int)
    # One correction step (r in [0, 2q))
    r = np.where(r >= q, r - q, r)
    r = np.where(r < 0, r + q, r)
    return r.astype(np.int64)

# ------------------------------
# Discrete Gaussian via CDT
# ------------------------------
@dataclass
class GaussianCDTTable:
    sigma: float
    tail_sigma: float = 10.0  # truncate at ~10*sigma by default

    def __post_init__(self):
        # Build nonnegative-side probabilities P[X = k] ∝ exp(-pi * k^2 / sigma^2)
        # for k >= 0 (we will add sign later; note that k=0 is unique).
        tmax = max(20, int(math.ceil(self.tail_sigma * self.sigma)))
        ks = np.arange(0, tmax + 1, dtype=np.int64)
        rho = np.exp(-math.pi * (ks.astype(np.float64) ** 2) / (self.sigma ** 2))
        # Normalize for nonnegative side (we sample k >= 0; sign handled after)
        Z = rho.sum()
        p = rho / Z  # P_nonneg[k]
        cdf = np.cumsum(p)
        cdf[-1] = 1.0  # guard against float drift
        self.ks = ks
        self.cdf = cdf

    def sample_nonneg(self, n: int, rng: np.random.Generator) -> np.ndarray:
        u = rng.random(n)
        idx = np.searchsorted(self.cdf, u, side="left")
        return self.ks[idx]

def sample_discrete_gaussian_cdt(n: int, sigma: float, tail_sigma: float = 10.0,
                                 rng: np.random.Generator | None = None) -> np.ndarray:
    """
    Two-sided discrete Gaussian via CDT on the nonnegative side + random sign.
    Returns integer samples with stddev approximately sigma (tail truncated).
    """
    if rng is None:
        rng = np.random.default_rng()
    table = GaussianCDTTable(sigma=sigma, tail_sigma=tail_sigma)
    k = table.sample_nonneg(n, rng)
    # random sign for k>0 (k=0 stays 0)
    s = rng.integers(0, 2, size=n, dtype=np.int8) * 2 - 1  # in {-1, +1}
    x = (k * s).astype(np.int64)
    x[k == 0] = 0
    return x

# ------------------------------
# Ternary sampler
# ------------------------------
def sample_ternary(n: int,
                   p_minus: float = 0.25,
                   p_zero: float = 0.50,
                   p_plus: float = 0.25,
                   rng: np.random.Generator | None = None) -> np.ndarray:
    """
    Ternary sampling for coefficients in {-1, 0, +1} with given probabilities.
    Probabilities must sum to 1. Vectorized.
    """
    if rng is None:
        rng = np.random.default_rng()
    if abs((p_minus + p_zero + p_plus) - 1.0) > 1e-12:
        raise ValueError("Probabilities must sum to 1.")
    edges = np.array([p_minus, p_minus + p_zero], dtype=np.float64)
    u = rng.random(n)
    out = np.empty(n, dtype=np.int8)
    out[:] = 1
    out[u < edges[0]] = -1
    out[(u >= edges[0]) & (u < edges[1])] = 0
    return out.astype(np.int8)

# ------------------------------
# Uniform mod-q sampler
# ------------------------------
def sample_uniform_mod_q(n: int, q: int, use_barrett: bool = False,
                         rng: np.random.Generator | None = None) -> np.ndarray:
    """
    Uniform residues in [0, q). The hardware typically uses Barrett; for
    Python we can use x % q or enable barrett for parity with RTL.
    """
    if rng is None:
        rng = np.random.default_rng()
    # Draw 64-bit randoms (as if from a PRNG/TRNG) and reduce mod q
    raw = rng.integers(0, 1 << 63, size=n, dtype=np.int64)
    if use_barrett:
        return barrett_reduce(raw, q)
    else:
        return (raw % q).astype(np.int64)

# ------------------------------
# Unified interface
# ------------------------------
class UnifiedSampler:
    """
    Unified sampler with three modes: 'uniform', 'ternary', 'gaussian'.
    Emulates a mode-selectable hardware sampler sharing one entropy source.
    """
    def __init__(self, seed: int | None = None):
        self.rng = np.random.default_rng(seed)

    def sample(self, mode: str, n: int,
               q: int | None = None,
               p_minus: float = 0.25, p_zero: float = 0.50, p_plus: float = 0.25,
               sigma: float = 3.2, tail_sigma: float = 10.0,
               use_barrett: bool = False) -> np.ndarray:
        mode = mode.lower()
        if mode == "uniform":
            if q is None:
                raise ValueError("Uniform mode requires modulus q.")
            return sample_uniform_mod_q(n, q, use_barrett=use_barrett, rng=self.rng)
        elif mode == "ternary":
            return sample_ternary(n, p_minus=p_minus, p_zero=p_zero, p_plus=p_plus, rng=self.rng)
        elif mode == "gaussian":
            return sample_discrete_gaussian_cdt(n, sigma=sigma, tail_sigma=tail_sigma, rng=self.rng)
        else:
            raise ValueError("Unknown mode. Use 'uniform', 'ternary', or 'gaussian'.")

# ------------------------------
# Example usage / quick sanity checks
# ------------------------------
if __name__ == "__main__":
    us = UnifiedSampler(seed=42)

    # 1) Uniform mod-q
    q = 7681
    u = us.sample("uniform", n=100_000, q=q)
    print("[Uniform] min,max =", int(u.min()), int(u.max()))
    print("[Uniform] unique coverage fraction ~", np.ptp(u.astype(np.int64)) / (q - 1))

    # 2) Ternary with 1:2:1
    t = us.sample("ternary", n=100_000, p_minus=0.25, p_zero=0.50, p_plus=0.25)
    p_m = (t == -1).mean()
    p_0 = (t == 0).mean()
    p_p = (t == +1).mean()
    print(f"[Ternary] P(-1)={p_m:.4f}, P(0)={p_0:.4f}, P(+1)={p_p:.4f}")

    # 3) Discrete Gaussian via CDT
    g = us.sample("gaussian", n=100_000, sigma=3.2, tail_sigma=10.0)
    print(f"[Gaussian] mean={g.mean():.4f}, std≈{g.std(ddof=0):.4f}")
    # crude symmetry check:
    print(f"[Gaussian] P(x=0)={np.mean(g==0):.4f}, P(x>0)={np.mean(g>0):.4f}, P(x<0)={np.mean(g<0):.4f}")


[Uniform] min,max = 0 7680
[Uniform] unique coverage fraction ~ 1.0
[Ternary] P(-1)=0.2522, P(0)=0.4981, P(+1)=0.2497
[Gaussian] mean=-0.0023, std≈1.1116
[Gaussian] P(x=0)=0.4750, P(x>0)=0.2614, P(x<0)=0.2636
